In [1]:
import pandas as pd
import numpy as np
import re

In [550]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.spatial.distance import cdist
from bs4 import BeautifulSoup
from  sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

In [3]:
def cleaner(text):
    
    text = re.sub(r'\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.lower()
    
    return text

In [4]:
def titles_extraction():
    
    titles_df = pd.read_csv('./data/docs_titles.tsv/docs_titles.tsv', sep='\t')
    docs_id_test = pd.read_csv('./data/test_groups.csv', sep=',')
    docs_id_train = pd.read_csv('./data/train_groups.csv', sep=',')

    info = pd.concat([docs_id_train, docs_id_test])
    info.reset_index(drop=True)

    titles = pd.merge(titles_df, info[['group_id', 'doc_id', 'target']], on='doc_id', how='inner')
    titles['title'] += ' '
    titles['title'].fillna(' ', inplace=True)

    titles = titles[['title', 'group_id']].groupby('group_id').sum()['title']

    titles = titles.apply(lambda x: cleaner(x))
    titles = titles.values

    return titles

In [65]:
all_titles = titles_extraction()
# all_titles

In [451]:
def bad_words(all_titles, threshold=0.015, group_number=None, ngram=(1,2)):
    
    X = TfidfVectorizer(ngram_range = ngram)
    importance = X.fit_transform(all_titles)
#     print(all_titles[group_number], group_number)
    if group_number is not None:
        
        group_titles = importance[group_number]
        group_titles = group_titles[0]
        
        new_dict = {v: k for k, v in X.vocabulary_.items()}
        
        array = np.where(np.bitwise_and(group_titles.toarray()!=0, group_titles.toarray()<threshold))[1]
#         print(len(group_titles.toarray()[np.where(np.bitwise_and(group_titles.toarray()!=0, group_titles.toarray()<threshold))]))
#         print('--------------',len([new_dict[index] for index in array]))
        return [new_dict[index] for index in array]
    
    else:

        group_bad_words = {group: bad_words(all_titles, threshold, group, ngram) for group in range(len(all_titles))}

        return group_bad_words

In [452]:
threshold = [0.01, 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.02]
for t in threshold:
    print(t)
    print(bad_words(all_titles, t, 1))

0.01
['ru как', 'архив страница', 'без', 'блог', 'за', 'записи', 'из', 'или', 'сайт']
0.011
['net', 'ru как', 'архив страница', 'без', 'блог', 'всё', 'группы', 'для', 'за', 'записи', 'из', 'или', 'мир', 'правильно', 'про', 'сайт']
0.012
['12', 'net', 'ru как', 'архив страница', 'без', 'блог', 'всё', 'глава', 'группы', 'детей', 'для', 'за', 'записи', 'записи рубрике', 'из', 'или', 'как правильно', 'когда', 'мир', 'помощь', 'правильно', 'про', 'рубрике', 'сайт', 'сообщения', 'центр']
0.013
['12', '14', '20', '2010', 'net', 'ru как', 'архив страница', 'без', 'блог', 'всё', 'газета', 'глава', 'группы', 'группы вконтакте', 'детей', 'для', 'за', 'записи', 'записи рубрике', 'из', 'из официальной', 'или', 'как правильно', 'карта', 'когда', 'материнство', 'мир', 'москва', 'новости сообщения', 'официальной', 'официальной группы', 'помощь', 'после', 'правильно', 'про', 'рубрике', 'сайт', 'сколько', 'сообщения', 'сообщения из', 'страна', 'страница', 'центр', 'через', 'что делать', 'яплакалъ']
0.01

In [37]:
corpus = [
        'This is the first document.',
        'This document is the second document.',
        'And this is the third one.',
        'Is this the first document?',
         ]

In [62]:
%%time

bad_words(corpus, 0.35)

CPU times: user 12.7 ms, sys: 2.92 ms, total: 15.6 ms
Wall time: 13.1 ms


{0: ['document', 'is', 'is the', 'the', 'this'],
 1: ['is', 'is the', 'the', 'this'],
 2: ['is', 'is the', 'the', 'this', 'this is'],
 3: ['document', 'first', 'first document', 'is', 'the', 'the first', 'this']}

In [232]:
%%time
words = bad_words(all_titles, 0.013)
# list(all_titles)
# плохие слова!!!

CPU times: user 4min 34s, sys: 9.61 s, total: 4min 43s
Wall time: 4min 44s


In [551]:
# for i in range(20):
#     print(len(np.unique(all_titles[i].split())), len(words[i]))

In [552]:
titles_df = pd.read_csv('./data/docs_titles.tsv/docs_titles.tsv', sep='\t')
docs_id_test = pd.read_csv('./data/test_groups.csv', sep=',')
docs_id_train = pd.read_csv('./data/train_groups.csv', sep=',')
print(len(docs_id_test) , len(docs_id_train))
info = pd.concat([docs_id_train, docs_id_test])
info.reset_index(drop=True)
print(len(info), len(titles_df))
titles = pd.merge(titles_df, info[['group_id', 'doc_id', 'target']], on='doc_id', how='right')
titles['title'].fillna(' ', inplace=True)
print(len(titles))
# titles[titles.title==' ']
# titles['group_id'].fillna(-1, inplace=True)
# titles[titles.group_id == -1 ]

16627 11690
28317 27950
28317


In [174]:
def group_titles_list_generator(df):
    
    for i in np.sort(df['group_id'].unique()):
        
        group_df = df[df.group_id == i]
        
        yield group_df

In [175]:
# Рассчет расстояний по матрице встречаемости

def count_distances(docs):
    
    distance = cdist(docs, docs, 'cityblock')
    
    res = np.asarray([np.concatenate((vec[:num],vec[num+1:])).astype(int) for num, vec in enumerate(distance)])
    res = np.sort(res)[:, :20]
#     res = np.flip(res, axis=1)[:, :20]
    
    return res

In [213]:
# Рассчитывает расстояния между документами берет 20 наименьших и сохраняет numpy ndarray в файл

def features_save(titles, vec_type=1):

    for group_titles in group_titles_list_generator(titles):
        
        group_num = group_titles['group_id'].unique()[0]
        group_titles = list(group_titles['title'])

        if vec_type == 1:
            vectorizer = CountVectorizer()
            X = vectorizer.fit_transform(group_titles)

        elif vec_type == 2:
            vectorizer2 = CountVectorizer()
            X = vectorizer2.fit_transform(group_titles)
        
        features = count_distances(X.toarray())
#         print('group number = {}, differ = {}'.format(group_num, len(group_titles) - features.shape[0]))

        np.save('group_features/{}'.format(group_num), features)
        print('Group number {} saved'.format(group_num))
    return features

In [214]:
def all_group_feature_list(start_group, finish_group):
    
    res = np.load('group_features/{}.npy'.format(start_group))
  
    for group_num in range(start_group + 1, finish_group + 1):
#         res += ndarray_to_list(np.load('group_features/{}.npy'.format(group_num)))
        res = np.vstack((res, np.load('group_features/{}.npy'.format(group_num))))
        
    return res

In [183]:
test = group_titles_list_generator(titles)

In [196]:
titles_df = pd.read_csv('./data/docs_titles.tsv/docs_titles.tsv', sep='\t')
len(titles_df), len(titles)

(27950, 28241)

In [215]:
features_save(titles)

Group number 1 saved
Group number 2 saved
Group number 3 saved
Group number 4 saved
Group number 5 saved
Group number 6 saved
Group number 7 saved
Group number 8 saved
Group number 9 saved
Group number 10 saved
Group number 11 saved
Group number 12 saved
Group number 13 saved
Group number 14 saved
Group number 15 saved
Group number 16 saved
Group number 17 saved
Group number 18 saved
Group number 19 saved
Group number 20 saved
Group number 21 saved
Group number 22 saved
Group number 23 saved
Group number 24 saved
Group number 25 saved
Group number 26 saved
Group number 27 saved
Group number 28 saved
Group number 29 saved
Group number 30 saved
Group number 31 saved
Group number 32 saved
Group number 33 saved
Group number 34 saved
Group number 35 saved
Group number 36 saved
Group number 37 saved
Group number 38 saved
Group number 39 saved
Group number 40 saved
Group number 41 saved
Group number 42 saved
Group number 43 saved
Group number 44 saved
Group number 45 saved
Group number 46 sav

array([[ 4,  4,  4, ...,  6,  6,  6],
       [ 4,  5,  5, ...,  9,  9,  9],
       [ 0,  0,  0, ...,  4,  4,  4],
       ...,
       [ 2,  2,  3, ...,  7,  7,  7],
       [ 0,  0,  2, ...,  7,  7,  7],
       [ 0,  0,  0, ..., 13, 13, 13]])

In [192]:
len(titles)

28241

In [238]:
def get_df():
    
    titles_df = pd.read_csv('./data/docs_titles.tsv/docs_titles.tsv', sep='\t')
    docs_id_test = pd.read_csv('./data/test_groups.csv', sep=',')
    docs_id_train = pd.read_csv('./data/train_groups.csv', sep=',')

    info = pd.concat([docs_id_train, docs_id_test])
    info.reset_index(drop=True)

    titles = pd.merge(titles_df, info[['group_id', 'doc_id', 'target']], on='doc_id', how='right')

    titles['title'].fillna(' ', inplace=True)
    
    return titles

In [239]:
def cleaning(title, group_num, bad_words):
    
    title = title.lower()
    title = re.sub(r'\W', '  ', title)
    
    for i in bad_words[group_num-1]:
            title = title.replace(' ' + i + ' ', ' ')
    title = re.sub(r'\s+', ' ', title)
    
    return title

In [244]:
def bad_words_cleaner(bad_words):
                            #dict: {group_1 : bad_words_1,
                            #       .....................
                            #       group_n : bad_words_n}     
    df = get_df()
    print(len(df))
    df['title'] = df[['title', 'group_id']].apply(lambda x: cleaning(*x, bad_words), axis=1)
    
    return df

In [553]:
# processed_df = bad_words_cleaner(words)
# processed_df[processed_df.target is not NaN]

In [554]:
# features_save(processed_df)

In [563]:
def upload_mised_titles():
    
    df = get_df()
    df['title'].fillna(' ', inplace=True)
    docs_id = df['doc_id'][df.title == ' '].unique()
    df = pd.read_csv('./data/docs_titles.tsv/docs_titles.tsv', sep='\t')
    df
    titles = []
    for i in docs_id:

        file = open('./content/' + str(i) + '.dat', 'r', encoding = 'utf-8')
        text = file.read()
        soup = BeautifulSoup(text, 'html')

        words = ''
        for title in soup.find_all(['title']):
            words += (title.text + ' ')
        titles.append(words)

    new_df = pd.DataFrame({'title': list(df['title']) + titles,
                           'doc_id': list(df['doc_id']) + list(docs_id)})
    
    new_df['title'].fillna(' ', inplace=True)
    shit = new_df['doc_id'][new_df.title == ' '].unique()
    
    docs_id_test = pd.read_csv('./data/test_groups.csv', sep=',')
    docs_id_train = pd.read_csv('./data/train_groups.csv', sep=',')

    info = pd.concat([docs_id_train, docs_id_test])
    info.reset_index(drop=True)

    titles = pd.merge(new_df, info[['group_id', 'doc_id', 'target']], on='doc_id', how='right')
    titles = titles[['doc_id', 'title', 'group_id', 'target']]

    titles['title'] = titles['title'].apply(lambda x: x.replace('\n', ' '))
    
    titles['title'] = stemming_titles(titles)
    
    titles.to_csv('./data/unversal_table.csv', index=False)
    
    return shit, titles

In [564]:
upload_mised_titles()

(array([17939, 27042, 27737, 26800,  4078,  8569,  2073, 11658, 17122,
        17485,  9164, 10590,  2952,  3002, 18840, 13623, 23594, 16795,
        13853, 10364, 15252,  3646,  7853, 12195,  5280,  4734, 20393,
        20572,   589, 14730, 16225, 21529]),
        doc_id                                              title  group_id  \
 0       15731            ваз 21213 | зам подшипник ступиц | нив          1   
 1       14829  ваз 2107 опт в сочи. сравн цены, куп потребите...         1   
 2       15764  куп ступиц лад калина2. трансмисс - переходн с...         1   
 3       17669                             классик 21010 - 21074          1   
 4       14852                ступиц нив — зам подшипник сво рук          1   
 ...       ...                                                ...       ...   
 28344   19232         боев техник - жизн и судьб ( втор выпуск)        267   
 28345   18594                               про установк обратк        267   
 28346   20015  арматур для уни

In [ ]:
# Загружаем готовое

In [532]:
doc_to_title = {}
with open('./data/unversal_table.csv') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
#         print(type(line))
        line = line.replace('\t', ',')
        
        data = line.strip().split(',')
        print(data)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print (len(doc_to_title))

# s = Porter

# for i in doc_to_title:
#     doc_to_title[i] = s.stem(doc_to_title[i])
# doc_to_title

['15731', 'ВАЗ 21213 | Замена подшипников ступицы | Нива', '1', '0.0']
['14829', '"Ваз 2107 оптом в Сочи. Сравнить цены', ' купить потребительские товары на Tiu.ru"', '1', '0.0']
['15764', '"Купить ступица Лада калина2. Трансмиссия - переходные ступицы цена', ' замена', ' тюнинг."', '1', '0.0']
['17669', 'Классика 21010 - 21074', '1', '0.0']
['14852', 'Ступица Нива — замена подшипника своими руками', '1', '0.0']
['15458', 'ВАЗ 2110', '1', '0.0']
['14899', '"Обзор подшипников полуоси ВАЗ 2101-07', ' 2121', '2123"', '1', '0.0']
['16879', 'Купить Подшипники и ступицы FAG (Страница 23)', '1', '0.0']
['16310', 'HorsePowers — автомобильный интернет портал » Отзыв владельца ВАЗ 2121 Нива 2007 года', '1', '0.0']
['15440', 'Новости и сообщения из официальной группы Вконтакте торговой компании 33 Sport - Магазины - Тольятти', '1', '0.0']
['16242', 'Инструкция по замене подшипника передней ступицы ивеко дейли через dorognoekam.ru', '1', '0.0']
['16383', 'Ступицы - OLX.ua - страница 80', '1', '0.0

['12242', 'Как создать образ диска с помощью UltraISO | Complandia. Обзор матчей', '21', '0.0']
['11472', 'VMware Workstation загрузка windows с флешки - Эмуляторы и виртуальные машины - CyberForum.ru', '21', '0.0']
['11703', 'Как сделать загрузочную флешку - YouTube', '21', '1.0']
['11421', 'Через Какую Программу Сделать Презентацию - YouTube', '21', '0.0']
['14036', 'Настройка прокси Android 3.2', '21', '0.0']
['12853', 'Как на ноутбуке hp загрузиться с флешки', '21', '0.0']
['12184', '"Как сделать загрузочную флешку с Windows 7', '8 !! Быстро!!!! - YouTube"', '21', '1.0']
['13975', 'Как изменить фон загрузчика у флешки? Урок №3 - YouTube', '21', '0.0']
['12698', 'Восстановление Windows с USB-флешки и DVD', '21', '0.0']
['13673', 'Новая жизнь ШГУ (Prestige/Premium). Прошивка на сторонние навигационные программы [Архив]  - Страница 7 - киа форум -  KIA Клуб', '21', '0.0']
['11784', '*****Alla Studentova***** : LiveInternet - Российский Сервис Онлайн-Дневников', '21', '0.0']
['14697', 

['26489', 'Народные рецепты для похудения', '42', '1.0']
['26972', 'Народные методы для сжигания жира на животе - Женский сайт «Катерина»', '42', '1.0']
['27281', 'Народные средства для похудения', '42', '1.0']
['26419', 'Быстрое похудение народные средства » Ваш доктор Айболит', '42', '1.0']
['26073', 'Как быстро похудеть в домашних условиях - Женский сайт «Катерина»', '42', '1.0']
['26084', 'Народные рецепты для похудения в домашних условиях', '42', '1.0']
['27722', 'Как вылечить кашель быстро в домашних условиях', '42', '0.0']
['25866', '"Народные средства для похудения', ' худеем народными средствами"', '42', '1.0']
['26695', '"Диеты для быстрого похудения-Лечение травами', ' рецепты народной медицины"', '42', '1.0']
['26037', 'Народные средства для похудения: как сбросить вес с помощью домашних способов', '42', '1.0']
['26813', 'Как-быстро-похудеть-в-домашних-условиях. Обсуждение на LiveInternet - Российский Сервис Онлайн-Дневников', '42', '1.0']
['26586', 'Слабительные средства в

['23390', 'юрок петров', '64', '0.0']
['22155', '"В магнитофоне кассета Челентано - слушать онлайн', ' быстро скачать без регистрации в формате mp3 только у нас"', '64', '0.0']
['23753', 'Автопонты... (21/28) [Форумы Balancer.Ru]', '64', '0.0']
['24276', 'Елена Попова. Проза — Рубрики автора — Омилия', '64', '0.0']
['24572', 'Lib.ru/Современная литература: Лещинский Леонид Абрамович. Бунт в умах', '64', '0.0']
['22474', 'ЯПлакалъ - Архив от 16.07.2014', '64', '0.0']
['24257', 'требования | Спортивные статьи - Part 11', '64', '0.0']
['22512', 'Скачать ай лайк - музыку в mp3 или слушать онлайн - найдено 281 песни/песен на Zvonko.me!', '64', '0.0']
['24127', '"как челентано рева музыка в MP3 - скачать бесплатно', ' слушать музыку как челентано рева - 14 песен/песни онлайн на Zaycev.online!"', '64', '1.0']
['22045', 'Ну хоть про кино ... :: FoxClub Forums  ::  Курилка', '64', '0.0']
['23169', '"Италия', ' кино и паста"', '64', '0.0']
['22694', 'Черная Луна • Астрологические консультации Фо

['25959', 'Мобильный форум Омска. Кодекс торговли. Разбор техники в Омске. Ремонт сотовых в Омске. Запчасти для сотовых. Интернет доска частных бесплатных объявлений электроники Мобильный Омск.', '79', '0.0']
['26842', 'Скачать квартиры для симс 3 бесплатно', '79', '0.0']
['27662', 'Где скачать The Sims 4 Малыши и Вампиры? Новый способ arkwars.ru', '79', '0.0']
['27391', 'Скачать моды на планшет в симс 3', '79', '0.0']
['27858', 'Видео-урок №1 Учимся скачивать доп.материалы для The sims 3. Уроки вязания на видео', '79', '0.0']
['842', '"Как установить Sims 4', ' а также как установить моды и дополнения к ней"', '79', '1.0']
['842', '"Как установить Sims 4', ' а также как установить моды и дополнения к ней"', '99', '1.0']
['25468', 'ДОКЛАД - Администрация Ярославской области', '79', '0.0']
['26181', 'Установка The Sims 3 + Выборочные дополнения. - Форум', '79', '0.0']
['26337', 'город где ты можешь делать все как гта как играть в гта 5 бес приставки игры гта 5 играть бес кадов кады на п

...

['5747', 'В России начнут применять новый вид уголовного наказания', '166', '']
['6536', 'Сергей Марченко | ВКонтакте', '166', '']
['6566', 'Леонтьев А. О новом издании первого тома &laquo;Капитала&raquo;', '166', '']
['7360', '"Как рассчитать «больничный»: формула', ' примеры"', '166', '']
['6282', 'КНИГА ВТОРАЯ Становление Учителя / Множественные умы Билли Миллигана', '166', '']
['6784', 'Setroi - Артефактор | samread', '166', '']
['6831', 'Как рассчитать отпускные в 2016 году: Инструкция', '166', '']
['4925', 'Как рассчитать среднедневной заработок: особенности и методика', '166', '']
['6232', 'дородовые', '166', '']
['4370', 'Как рассчитать компенсацию при увольнении - расчет компенсации', '166', '']
['15344', '"И снова о моторе - 1', '6 vs 1', '4TSI [Архив]  - Страница 5 - Фольксваген Гольф Клуб"', '167', '']
['17805', 'АКПП Киа Рио A4CF1 Kia Rio 2012 - Эксплуатация автомобиля - Страница 5 из 8 - Клуб владельцев Kia Rio Санкт-Петербург', '167', '']
['17581', 'Chevrolet Epica Клуб 

['22224', 'Супергерои The CW вышли на битву с пришельцами в трейлере кроссовера | Новости  | Канобу', '188', '']
['22886', '"сериал флэш 3 серия - hd', ' в хорошем качестве', ' видеобокс.тв', ' смотреть онлайн - VideoBox.tv"', '188', '']
['23146', 'MySQL Fatal Error', '188', '']
['23347', '"Супергёрл 3 сезон дата выхода всех серий | Дата выхода фильмов', 'игр и сериалов."', '188', '']
['22779', 'Смотреть онлайн фильмы в хорошем качестве в 720p hd', '188', '']
['22542', 'Смотреть хорошее кино онлайн на KinoVO.cc » Страница 810', '188', '']
['22844', 'Скачать Люцифер 2 сезон / Lucifer (2016) MP4 бесплатно с торрента без регистрации - Фильмы  - TorrentBest.ru - скачать бесплатно торренты без регистрации - Торрент трекер портал', '188', '']
['23609', '"Форум', ' Списко сообщений темы"', '188', '']
['24689', 'Вампир-прокурор 2 / Vampire Prosecutor 2  [2012] - Смотреть On-Line', '188', '']
['21674', '"СуперГерл дата выхода серий 3 сезона', ' новости"', '188', '']
['24118', 'Материнство > Я-С

['23035', 'Ответы@Mail.Ru: Как готовить стерлядь???', '208', '']
['23711', '"как приготовить стерлядь в духовке целиком » ВКУСНЫЕ РЕЦЕПТЫ С ФОТО. Что приготовить на завтрак', ' на обед', ' на ужин. Красивые кулинарные рецепты на каждый день!"', '208', '']
['22309', 'Как готовить стерлядь в духовке » ВКУСНЫЕ РЕЦЕПТЫ 2017 С ФОТО', '208', '']
['23696', 'Стерлядь запеченная - пошаговый рецепт с фото - для духовки.', '208', '']
['23822', 'Осетр целиком в духовке рецепт | Как приготовить осетра в духовке', '208', '']
['23671', 'Запекаем осетра в духовке целиком пошаговый рецепт с фото', '208', '']
['22258', 'Как приготовить стерлядь в духовке | Блог о вкусной и здоровой пище', '208', '']
['23977', 'Как приготовить осетра в духовке: советы и рецепты :: SYL.ru', '208', '']
['24243', 'Стерлядь в фольге в духовке - пошаговый рецепт с фото на Повар.ру', '208', '']
['22108', 'Как запекать стерлядь. При какой температуре запечь стерлядь. Рецепт запеченной стерляди в фольге. Как запечь рыбу стерлядь

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




['7516', 'Скачать Скачать бесплатно PDF-XChange Viewer PRO 2.5.205 (2012) ML/RUS+ключ - Текстовые редакторы  - программы на русском скачать новый софт - RUS SOFT', '304', '']
['4054', 'Журнал «Компьютерра» № 25-26 от 10 июля 2007 года (693 и 694 номер) (fb2) | КулЛиб - Классная библиотека! Скачать книги бесплатно', '304', '']
['4473', 'Ландшафтный дизайн: Проектирование домов и коттеджей (Программы)', '304', '']
['5114', 'Android тень текста', '304', '']
['5756', 'Word 2007 : Информатика : ДИПЛОМНАЯ - БАЗА ДИПЛОМНЫХ РАБОТ РОССИЙСКИХ ВУЗОВ.', '304', '']
['5501', 'Конвертировать в BMP', '304', '']
['5792', 'PDF в BMP: конвертировать из PDF в BMP (битовые изображения)', '304', '']
['4356', 'Регистрация total pdf converter — chinashops.ru', '304', '']
['5443', 'Конвертировать изображения JPG JPEG PNG BMP GIF TIF TIFF в PDF Онлайн Бесплатно', '304', '']
['4564', 'Конвертация PDF в BMP с помощью Фотоконвертера', '304', '']
['6230', '"ABViewer 9 Enterprise 9.0.0.5 x86 [2013', ' MULTILANG +RU

In [350]:
def tuple_x(a1, a2):
    return a1, a2

In [557]:
def my_doc_to_title():
    
    processed_df = get_df()
    processed_df['title'] = stemming_titles(processed_df)
    processed_df
    
    processed_df['target'].fillna(-1, inplace=True)
    train_titles = processed_df[processed_df.target!=-1]
    test_titles = processed_df[processed_df.target==-1]
    
    processed_df['tuple'] = processed_df[['doc_id', 'title']].apply(lambda x: tuple_x(*x), axis=1)
    doc_to_title = {}
    for i in processed_df['tuple']:
        doc_to_title[i[0]] = i[1]
        
    return doc_to_title

In [555]:

# train_titles, test_titles

In [558]:
doc_to_title = my_doc_to_title()

In [559]:
import pandas as pd
train_data = pd.read_csv('./data/train_groups.csv')
# train_data = train_titles
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))
# traingroups_titledata

In [535]:
import numpy as np
y_train = []
X_train = []
groups_train = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_train.append(sorted(all_dist, reverse=True)[0:25]    )
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train.shape, y_train.shape, groups_train.shape)

(11690, 25) (11690,) (11690,)


In [560]:
import pandas as pd
test_data = pd.read_csv('./data/test_groups.csv')
# test_data = test_titles
traingroups_titledata = {}
for i in range(len(test_data)):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title))
# traingroups_titledata

In [537]:
# y_train = []
X_test = []
groups_test = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title) in enumerate(docs):
#         y_train.append(target_id)
        groups_test.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_test.append(sorted(all_dist, reverse=True)[0:25]    )
X_test = np.array(X_test)
# y_train = np.array(y_train)
groups_test = np.array(groups_test)
print (X_test.shape, groups_test.shape)

(16627, 25) (16627,)


In [538]:
clf = GradientBoostingClassifier(learning_rate=0.06, n_estimators=150)

In [539]:
def predict(X_train, X_test, train_target, model, scaler=None, **kwargs):
    
    curr_model = model(**kwargs)
    
    if scaler is not None:
        
        your_scaler = scaler()
        your_scaler.fit(X_train)
        X_train = your_scaler.transform(X_train)
        X_test = your_scaler.transform(X_test)
        
    curr_model.fit(X_train, train_target)
#     y_pred = curr_model.predict(X_test)
    th = 0.34
    y_pred = [0 if val < th else 1 for val in curr_model.predict_proba(X_test)[:,1]]
    return y_pred

In [546]:
# Сохраняет решение

def save_submission(y_pred):

    data = pd.read_csv('data/test_groups.csv')
    print('len data = ', len(data))
    data['target'] = y_pred
    
#     for x in shit:
#         print(data[data.doc_id==x]['target'])
    
    data = data.drop(['group_id', 'doc_id'], axis=1)

    data.to_csv("submission.csv", index=False)
    
    info = np.unique(data['target'], return_counts=True)
    
    if info[0].shape[0] > 1:
        
        print('0: {}, 1: {}'.format(info[1][0], info[1][1]))
        if info[1][1] > 3600 or info[1][1] < 2500:
            print('Your submisson is shit')
    
    else:
        print('There are only {} in submission'.format(info[0][0]))
        
    return data

In [547]:
y_pred = predict(X_train, X_test, y_train, GradientBoostingClassifier, learning_rate=0.06, n_estimators=150)
data = save_submission(y_pred)

len data =  16627
Series([], Name: target, dtype: int64)
Series([], Name: target, dtype: int64)
Series([], Name: target, dtype: int64)
Series([], Name: target, dtype: int64)
Series([], Name: target, dtype: int64)
Series([], Name: target, dtype: int64)
Series([], Name: target, dtype: int64)
Series([], Name: target, dtype: int64)
Series([], Name: target, dtype: int64)
Series([], Name: target, dtype: int64)
Series([], Name: target, dtype: int64)
Series([], Name: target, dtype: int64)
Series([], Name: target, dtype: int64)
Series([], Name: target, dtype: int64)
Series([], Name: target, dtype: int64)
Series([], Name: target, dtype: int64)
671    0
Name: target, dtype: int64
2853    0
Name: target, dtype: int64
3006    0
Name: target, dtype: int64
4046    0
Name: target, dtype: int64
6027    0
Name: target, dtype: int64
6403     0
16320    0
Name: target, dtype: int64
6913    0
Name: target, dtype: int64
9183    0
Name: target, dtype: int64
11467    0
Name: target, dtype: int64
11482    0
Na

In [417]:
import re

class Porter:
    PERFECTIVEGROUND =  re.compile(u"((ив|ивши|ившись|ыв|ывши|ывшись)|((?<=[ая])(в|вши|вшись)))$")
    REFLEXIVE = re.compile(u"(с[яь])$")
    ADJECTIVE = re.compile(u"(ее|ие|ые|ое|ими|ыми|ей|ий|ый|ой|ем|им|ым|ом|его|ого|ему|ому|их|ых|ую|юю|ая|яя|ою|ею)$")
    PARTICIPLE = re.compile(u"((ивш|ывш|ующ)|((?<=[ая])(ем|нн|вш|ющ|щ)))$")
    VERB = re.compile(u"((ила|ыла|ена|ейте|уйте|ите|или|ыли|ей|уй|ил|ыл|им|ым|ен|ило|ыло|ено|ят|ует|уют|ит|ыт|ены|ить|ыть|ишь|ую|ю)|((?<=[ая])(ла|на|ете|йте|ли|й|л|ем|н|ло|но|ет|ют|ны|ть|ешь|нно)))$")
    NOUN = re.compile(u"(а|ев|ов|ие|ье|е|иями|ями|ами|еи|ии|и|ией|ей|ой|ий|й|иям|ям|ием|ем|ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я)$")
    RVRE = re.compile(u"^(.*?[аеиоуыэюя])(.*)$")
    DERIVATIONAL = re.compile(u".*[^аеиоуыэюя]+[аеиоуыэюя].*ость?$")
    DER = re.compile(u"ость?$")
    SUPERLATIVE = re.compile(u"(ейше|ейш)$")
    I = re.compile(u"и$")
    P = re.compile(u"ь$")
    NN = re.compile(u"нн$")

    def stem(string):
        
        changed = ''
        
        string = string.lower()

        for word in string.split():
            if not word.isdigit():
                word = word.replace(u'ё', u'е')
                m = re.match(Porter.RVRE, word)

                if m and m.groups():
                    pre = m.group(1)
                    rv = m.group(2)
                    temp = Porter.PERFECTIVEGROUND.sub('', rv, 1)
                    if temp == rv:
                        rv = Porter.REFLEXIVE.sub('', rv, 1)
                        temp = Porter.ADJECTIVE.sub('', rv, 1)
                        if temp != rv:
                            rv = temp
                            rv = Porter.PARTICIPLE.sub('', rv, 1)
                        else:
                            temp = Porter.VERB.sub('', rv, 1)
                            if temp == rv:
                                rv = Porter.NOUN.sub('', rv, 1)
                            else:
                                rv = temp
                    else:
                        rv = temp

                    rv = Porter.I.sub('', rv, 1)

                    if re.match(Porter.DERIVATIONAL, rv):
                        rv = Porter.DER.sub('', rv, 1)

                    temp = Porter.P.sub('', rv, 1)
                    if temp == rv:
                        rv = Porter.SUPERLATIVE.sub('', rv, 1)
                        rv = Porter.NN.sub(u'н', rv, 1)
                    else:
                        rv = temp
                    word = pre+rv

            changed += word + ' '

        return changed

#     stem=staticmethod(stem)

#     if __name__ == '__main__':
#         print Porter.stem(u'устойчивость')

In [414]:
Porter.stem(u'игроки играли в скоростные мячи')

'игрок игра в скоростн мяч '

In [415]:
def stemming_titles(df):
    stemming = Porter
    return df['title'].apply(lambda x: stemming.stem(x))

In [419]:
processed_df['title'] = stemming_titles(processed_df)
processed_df

,doc_id,title,group_id,target,tuple,new_title
0,15731,ваз 21213 зам подшипник ступиц нив,1,0.0,"(15731, ваз 21213 замена подшипников ступицы н...",ваз зам подшипник ступиц нив
1,14829,ваз 2107 в соч сравн цен куп потребительск тов...,1,0.0,"(14829, ваз 2107 в сочи сравнить цены купить п...",ваз соч сравн цен куп потребительск товар на
2,15764,куп ступиц лад калина2 переходн ступиц цен зам,1,0.0,"(15764, купить ступица лада калина2 переходные...",куп ступиц лад калина2 переходн ступиц цен зам
3,17669,классик 21010 21074,1,0.0,"(17669, классика 21010 21074 )",классик
4,14852,ступиц нив зам подшипник сво рук,1,0.0,"(14852, ступица нива замена подшипника своими ...",ступиц нив зам подшипник сво рук
...,...,...,...,...,...,...
28312,19232,,267,-1.0,"(19232, )",
28313,18594,,267,-1.0,"(18594, )",
28314,20015,,267,-1.0,"(20015, )",
28315,21264,,267,-1.0,"(21264, )",
